# Build a simulation using the Python API

Currently, this notbook manipulates the simulation by directly placing objects inside of the attributes of the network and domain. It should be refactored when proper methods exist for adding these objects.


Import the Simulation class

In [119]:
from primaite.simulator.sim_container import Simulation


Create an empty simulation. By default this has a network with no nodes or links, and a domain controller with no accounts.

Let's use the simulation's `describe_state()` method to verify that it is empty.

In [120]:
my_sim = Simulation()
net = my_sim.network
my_sim.describe_state()

{'uuid': '42d005b2-4dc8-4aec-be54-3493242eee32',
 'network': {'uuid': '069f61a4-ac40-431f-ad13-2fc9b26dc091',
  'nodes': {},
  'links': {}},
 'domain': {'uuid': 'f0629156-e9af-493d-b098-f47d73126122', 'accounts': {}}}

## Add nodes

In [121]:
from primaite.simulator.network.hardware.nodes.host.computer import Computer
from primaite.simulator.network.hardware.nodes.host.server import Server

In [122]:
my_pc = Computer(hostname="primaite_pc", ip_address="192.168.1.10", subnet_mask="255.255.255.0")
net.add_node(my_pc)
my_server = Server(hostname="google_server", ip_address="192.168.1.11", subnet_mask="255.255.255.0")
net.add_node(my_server)


## Connect the nodes

In [123]:
from primaite.simulator.network.hardware.nodes.host.host_node import NIC
from primaite.simulator.network.hardware.nodes.network.switch import Switch


In [124]:
my_switch = Switch(hostname="switch1", num_ports=12)
net.add_node(my_switch)

pc_nic = NIC(ip_address="130.1.1.1", gateway="130.1.1.255", subnet_mask="255.255.255.0")
my_pc.connect_nic(pc_nic)

server_nic = NIC(ip_address="130.1.1.2", gateway="130.1.1.255", subnet_mask="255.255.255.0")
my_server.connect_nic(server_nic)

net.connect(pc_nic, my_switch.network_interface[1])
net.connect(server_nic, my_switch.network_interface[2])


Link(uuid='42b8f911-3640-4ccb-b277-b48b294a1fc8', endpoint_a=NIC(ip_address=IPv4Address('130.1.1.2'), subnet_mask=IPv4Address('255.255.255.0'), uuid='53993d8f-216e-4c00-9b03-c6bb9e2437b5', mac_address='17:9d:82:db:ca:c8', speed=100, mtu=1500, enabled=False, port_num=2, port_name=None, pcap=None, nmne={}, wake_on_lan=False, gateway='130.1.1.255'), endpoint_b=SwitchPort(uuid='c03d4d22-f309-49b6-a1ad-45a04c40d25e', mac_address='84:01:f3:bb:47:1c', speed=100, mtu=1500, enabled=False, port_num=2, port_name=None, pcap=None, nmne={}), bandwidth=100.0, current_load=0.0)

## Add files and folders to nodes


In [125]:
from primaite.simulator.file_system.file_type import FileType
from primaite.simulator.file_system.file_system import File
from primaite.simulator.system.core.sys_log import SysLog

In [126]:
my_pc_downloads_folder = my_pc.file_system.create_folder("downloads")
my_pc_downloads_folder.add_file(File(name="firefox_installer.zip",folder_id="Test", folder_name="downloads" ,file_type=FileType.ZIP, sys_log=SysLog(hostname="Test")))

In [127]:
my_server_folder = my_server.file_system.create_folder("static")
my_server.file_system.create_file("favicon.ico", file_type=FileType.PNG)

File(uuid='24789051-6762-48f4-8a56-c28882374273', name='favicon.ico', health_status=<FileSystemItemHealthStatus.GOOD: 1>, visible_health_status=<FileSystemItemHealthStatus.GOOD: 1>, previous_hash=None, revealed_to_red=False, sys_log=<primaite.simulator.system.core.sys_log.SysLog object at 0x000002923D79AC50>, deleted=False, folder_id='7a86576b-607f-468b-826f-4834cf2b3511', folder_name='root', file_type=<FileType.UNKNOWN: 0>, sim_size=0, real=False, sim_path=None, sim_root=WindowsPath('C:/Projects/PrimAITE/simulation_output/2024-04-08_12-19-36/google_server/fs'), num_access=0, folder=Folder(uuid='7a86576b-607f-468b-826f-4834cf2b3511', name='root', health_status=<FileSystemItemHealthStatus.GOOD: 1>, visible_health_status=<FileSystemItemHealthStatus.GOOD: 1>, previous_hash=None, revealed_to_red=False, sys_log=<primaite.simulator.system.core.sys_log.SysLog object at 0x000002923D79AC50>, deleted=False, files={'24789051-6762-48f4-8a56-c28882374273': File(uuid='24789051-6762-48f4-8a56-c288823

## Add applications to nodes

In [128]:
from primaite.simulator.system.applications.application import Application, ApplicationOperatingState
from primaite.simulator.system.software import SoftwareHealthState, SoftwareCriticality
from primaite.simulator.network.transmission.transport_layer import Port
from primaite.simulator.network.transmission.network_layer import IPProtocol
from primaite.simulator.file_system.file_system import FileSystem
from pathlib import Path

# no applications exist yet so we will create our own.
class MSPaint(Application):
    def describe_state(self):
        return super().describe_state()

In [129]:
mspaint = MSPaint(name = "mspaint", health_state_actual=SoftwareHealthState.GOOD, health_state_visible=SoftwareHealthState.GOOD, criticality=SoftwareCriticality.MEDIUM, port=Port.HTTP, protocol = IPProtocol.NONE,operating_state=ApplicationOperatingState.RUNNING,execution_control_status='manual', file_system=FileSystem(sys_log=SysLog(hostname="Test"), sim_root=Path(__name__).parent),)

In [130]:
my_pc.applications[mspaint.uuid] = mspaint

## Create a domain account

In [131]:
from primaite.simulator.domain.account import Account, AccountType


In [132]:
acct = Account(username="admin", password="admin12", account_type=AccountType.USER)
my_sim.domain.accounts[acct.uuid] = acct

Verify that the state dictionary contains no non-serialisable objects.

In [133]:
my_sim.describe_state()

{'uuid': '42d005b2-4dc8-4aec-be54-3493242eee32',
 'network': {'uuid': '069f61a4-ac40-431f-ad13-2fc9b26dc091',
  'nodes': {'primaite_pc': {'uuid': '52246eed-9a3f-4b19-ad0c-48fc3bbb998d',
    'hostname': 'primaite_pc',
    'operating_state': 2,
    'NICs': {1: {'uuid': '73dcb42e-7db4-45cf-b439-9b8066c8e32e',
      'mac_address': 'c9:84:ec:48:87:77',
      'speed': 100,
      'mtu': 1500,
      'enabled': False,
      'nmne': {},
      'ip_address': '192.168.1.10',
      'subnet_mask': '255.255.255.0',
      'wake_on_lan': False},
     2: {'uuid': 'e0fbda66-afcb-4a79-b696-aad0778279a2',
      'mac_address': 'cb:66:8b:b2:dc:51',
      'speed': 100,
      'mtu': 1500,
      'enabled': False,
      'nmne': {},
      'ip_address': '130.1.1.1',
      'subnet_mask': '255.255.255.0',
      'wake_on_lan': False}},
    'file_system': {'uuid': '8a857927-dd5e-40e9-86fd-1df8b3a2b463',
     'folders': {'root': {'uuid': 'acb725c9-461e-40c5-b2c0-ed198865e1f2',
       'name': 'root',
       'health_statu

In [134]:
import json
json.dumps(my_sim.describe_state())

'{"uuid": "42d005b2-4dc8-4aec-be54-3493242eee32", "network": {"uuid": "069f61a4-ac40-431f-ad13-2fc9b26dc091", "nodes": {"primaite_pc": {"uuid": "52246eed-9a3f-4b19-ad0c-48fc3bbb998d", "hostname": "primaite_pc", "operating_state": 2, "NICs": {"1": {"uuid": "73dcb42e-7db4-45cf-b439-9b8066c8e32e", "mac_address": "c9:84:ec:48:87:77", "speed": 100, "mtu": 1500, "enabled": false, "nmne": {}, "ip_address": "192.168.1.10", "subnet_mask": "255.255.255.0", "wake_on_lan": false}, "2": {"uuid": "e0fbda66-afcb-4a79-b696-aad0778279a2", "mac_address": "cb:66:8b:b2:dc:51", "speed": 100, "mtu": 1500, "enabled": false, "nmne": {}, "ip_address": "130.1.1.1", "subnet_mask": "255.255.255.0", "wake_on_lan": false}}, "file_system": {"uuid": "8a857927-dd5e-40e9-86fd-1df8b3a2b463", "folders": {"root": {"uuid": "acb725c9-461e-40c5-b2c0-ed198865e1f2", "name": "root", "health_status": 1, "visible_status": 1, "previous_hash": null, "revealed_to_red": false, "files": {}, "deleted_files": {}}, "downloads": {"uuid": 